In [9]:
import keras
from keras import initializers
from keras.optimizers import SGD
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, concatenate, Input, BatchNormalization, AveragePooling2D, Dropout,Dense
from keras.callbacks import LearningRateScheduler


In [10]:
def inception_module(x, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5,\
     filters_pool_proj, kernel_init, bias_init, name=None):
    
    # conv 1x1
    conv_1x1 = Conv2D(filters_1x1, kernel_size=(1,1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)

    # conv 1x1 + conv3x3
    pre_conv_3x3 = Conv2D(filters_3x3_reduce, kernel_size=(1,1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_3x3 = Conv2D(filters_3x3, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pre_conv_3x3)

    # conv 1x1 + conv5x5
    pre_conv_5x5 = Conv2D(filters_5x5_reduce, kernel_size=(1,1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_5x5 = Conv2D(filters_5x5, kernel_size=(5,5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pre_conv_3x3)

    # pool + conv 1x1
    pool_proj=MaxPooling2D((3,3), strides=(1,1), padding='same')(x)
    pool_proj=Conv2D(filters_pool_proj, kernel_size=(1,1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)

    output=concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    return output

In [13]:
def googLeNet():
    kernel_init = initializers.glorot_uniform()
    bias_init = initializers.Constant(value=0.2)

    # part A : 
    input_layer=Input((224,224,3))
    x=Conv2D(64, (7,7), padding='same', strides=(2,2), activation='relu', name='conv_1', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)

    x=MaxPooling2D((3,3), strides=(2,2), padding='same', name='max_pool_2')(x)

    x= BatchNormalization()(x)

    x=Conv2D(64, (1,1), (1,1), 'same', activation='relu')(x)
    x=Conv2D(192, (3,3), (1,1), 'same', activation='relu')(x)
    x= BatchNormalization()(x)
    x=MaxPooling2D((3,3), (2,2), 'same')(x)

    # part B
    x=inception_module(x, 64, 96, 128, 16, 32, 32, kernel_init, bias_init, name='inception_3a')
    x=inception_module(x, 128, 128, 192, 32, 96, 64, kernel_init, bias_init, name='inception_3b')

    x=inception_module(x, 192, 96, 208, 16, 48, 64, kernel_init, bias_init, name='inception_4a')
    x=inception_module(x, 160, 112, 224, 24, 64, 64, kernel_init, bias_init, name='inception_4b')
    x=inception_module(x, 128, 128, 256, 24, 64, 64, kernel_init, bias_init, name='inception_4c')
    x=inception_module(x, 112, 144, 288, 32, 64, 64, kernel_init, bias_init, name='inception_4d')
    x=inception_module(x, 256, 160, 320, 32, 128, 128, kernel_init, bias_init, name='inception_4e')
    
    x= MaxPooling2D((3,3), (2,2), 'same', name='max_pool_4')(x)

    x=inception_module(x, 256, 160, 320, 32, 128, 128, kernel_init, bias_init, name='inception_5a')
    x=inception_module(x, 384, 192, 384, 48, 128, 128, kernel_init, bias_init, name='inception_5b')

    # part C
    x=AveragePooling2D((7,7), 1, 'valid')(x)
    x=Dropout(0.4)(x)
    output=Dense(10, 'softmax', name='output')(x)

    model = Model(inputs=input_layer, outputs=output)

    return model

In [14]:
import math
# train

epochs =25
initial_lrate =0.01

def decay(epoch, steps=100):
    initial_lrate=0.01
    drop=0.96
    epochs_drop=8

    lrate=initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

lr_schedule = LearningRateScheduler(decay, verbose=1)

sgd= SGD(lr=initial_lrate, momentum=0.9, nesterov=False)

model=googLeNet()
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])